In [3]:
import os
import numpy as np

import LMCE.cfusdlog as cfusdlog
import LMCE.uav_trajectory as uav_trajectory
from LMCE.error_estimation import error_calculator, find_best_cutoff

Here we compare the different average errors for each flight. The flights have a timescale of 0.6.

In [12]:
# Get desired path
traj = uav_trajectory.Trajectory()
traj.loadcsv("./LMCE/flight_paths/figure8.csv")
traj.stretchtime(2)

ts = np.arange(0, traj.duration, 0.01)
evals = np.empty((len(ts), 15))
for t, i in zip(ts, range(0, len(ts))):
    e = traj.eval(t)
    e.pos += np.array([0, 0, 1])
    evals[i, 0:3] = e.pos

target_pos = evals.transpose()

main_path = "./results_flight_data/timescale7/cf21/"
categories = ["lee", "nn", "standard", "tree"]

for c in categories:
    avg_errors = []
    avg_it = []
    path = main_path + c
    for filename in os.listdir(path):
        if filename == "nn_log12":
            continue

        data_path = path + "/" + filename
        # Get real path
        data = cfusdlog.decode(data_path)['fixedFrequency']
        real_pos = [data["stateEstimate.x"], data["stateEstimate.y"]]

        # Calculate error
        cutoff = find_best_cutoff(real_pos, target_pos)
        error = error_calculator(cutoff, real_pos, target_pos, vis=False)
        # print(filename, error)
        avg_errors.append(error)
        avg_it.append(sum(data["nn_perf.inf_tim"])/len(data["nn_perf.inf_tim"]))

    avg_error = sum(avg_errors[:7])/len(avg_errors[:7])
    avg_it = sum(avg_it)/len(avg_it)
    print("----------------------------------------")
    print(f"Avg. error for '{c}': {avg_error:.5f} (meters) for the invidual flights: {avg_errors}")
    print(f"Avg. inf time for '{c}': {avg_it:.5f} (microseconds)")


----------------------------------------
Avg. error for 'lee': 0.09718 (meters) for the invidual flights: [0.10030853331073318, 0.0913724882598542, 0.09703414684600098, 0.09550155101281302, 0.09966726885731106, 0.09465300119863551, 0.10171717006914692, 0.09376660080959233]
Avg. inf time for 'lee': 0.00000 (microseconds)
----------------------------------------
Avg. error for 'nn': 0.09658 (meters) for the invidual flights: [0.10162363385022954, 0.09519702745611758, 0.0997083905458508, 0.09458636220622636, 0.0942323587030841, 0.0959373854676506, 0.0947495044238876, 0.0989011543202263]
Avg. inf time for 'nn': 18013.78181 (microseconds)
----------------------------------------
Avg. error for 'standard': 0.09801 (meters) for the invidual flights: [0.09967773498208557, 0.09637465154396123, 0.09652856385683685, 0.10145738076644367, 0.10252250346778471, 0.0914242704270875, 0.0980810507752353]
Avg. inf time for 'standard': 2.34403 (microseconds)
----------------------------------------
Avg. er